<a href="https://colab.research.google.com/github/EdenKantor/Cloud-Computing-Course-Tut/blob/main/Cloud_Tut3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# שליפת הנתונים מהמאגר
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3&limit=10000'
response = requests.get(url)
data = response.json()

# יצירת DataFrame מהנתונים
data_df = pd.DataFrame(data['result']['records'])

# יצירת רשימה ייחודית של יצרנים
unique_makers = data_df['tozeret_nm'].dropna().unique()

# יצירת תיבת בחירה עבור Tozeret (יצרן)
tozeret_dropdown = widgets.Dropdown(
    options=unique_makers,
    description='Tozeret:',
    disabled=False
)

# יצירת תיבת בחירה עבור Kinuy Mishari (דגם)
kinuy_mishari_dropdown = widgets.Dropdown(
    description='Kinuy Mishari:',
    disabled=False,
    style={'description_width': 'initial'}  # משפר את הצגת השם המלא
)

# יצירת תיבת בחירה עבור Ramat Gimur (רמות גימור)
ramat_gimur_output = widgets.Output()

# פונקציה לעדכון הדגמים לפי היצרן שנבחר
def update_models(change):
    chosen_maker = tozeret_dropdown.value
    # סינון הדגמים על פי היצרן
    models_by_maker = data_df[data_df['tozeret_nm'] == chosen_maker]['kinuy_mishari'].dropna().unique()
    kinuy_mishari_dropdown.options = models_by_maker
    # עדכון רמות הגימור גם כן
    update_trim_levels(None)

# פונקציה לעדכון רמות הגימור לפי הדגם שנבחר
def update_trim_levels(change):
    # ניקוי תצוגה קודמת
    clear_output(wait=True)
    display(tozeret_dropdown)
    display(kinuy_mishari_dropdown)

    # סינון המידע לפי היצרן והדגם שנבחרו
    chosen_maker = tozeret_dropdown.value
    chosen_model = kinuy_mishari_dropdown.value

    # אם לא נבחר דגם או יצרן, לא מציגים כלום
    if not chosen_maker or not chosen_model:
        return

    # סינון הנתונים על פי היצרן והדגם שנבחרו
    filtered_data = data_df[(data_df['tozeret_nm'] == chosen_maker) & (data_df['kinuy_mishari'] == chosen_model)]

    if filtered_data.empty:
        display(widgets.HTML("<b>No data available for the selected model.</b>"))
        return

    # רשימת רמות הגימור הייחודיות לדגם (ללא כפילויות)
    trim_levels = filtered_data['ramat_gimur'].dropna().unique()

    # הצגת רמות הגימור בפורמט של מערך (ללא רווחים)
    trim_levels_str = "[" + ", ".join(f"'{trim}'" for trim in trim_levels) + "]"

    with ramat_gimur_output:
        display(widgets.HTML("<b>Ramat Gimur:</b>"))
        display(widgets.HTML(trim_levels_str))  # הצגת רמות הגימור כמערך

    # הצגת מידע נוסף
    total_records = filtered_data.shape[0]
    unique_trim_levels_str = ", ".join(trim_levels)  # הצגת רמות הגימור כטקסט, לא כמספר
    display(widgets.HTML(f"<b>Total Records:</b> {total_records}"))
    display(widgets.HTML(f"<b>Unique Ramat Gimur:</b> {unique_trim_levels_str}"))

# הצגת התוצאות
tozeret_dropdown.observe(update_models, names='value')  # עדכון הדגמים לפי היצרן
kinuy_mishari_dropdown.observe(update_trim_levels, names='value')  # עדכון רמות הגימור לפי הדגם

# הצגת התיבות
display(tozeret_dropdown)
display(kinuy_mishari_dropdown)
display(ramat_gimur_output)


Dropdown(description='Tozeret:', index=6, options=('מזדה יפן', 'קיה קוריאה', 'יונדאי קוריאה', 'יונדאי טורקיה',…

Dropdown(description='Kinuy Mishari:', index=3, options=('GLE400D 4MATIC', 'GLE400D COUPE', 'ML350 4MATIC', 'M…

HTML(value='<b>Total Records:</b> 3')

HTML(value='<b>Unique Ramat Gimur:</b> , LUXURY')